In [1]:
import scripro
import anndata
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import scanpy as sc
import warnings
warnings.filterwarnings("ignore")

# Import Data

## Obtain the supercell division of each cell from RNA data

In [ ]:
atac = sc.read_h5ad('/fs/home/xuyunfan/project/scripmulti/mira/atac_scrip.h5ad')

In [2]:
rna = sc.read_h5ad("/fs/home/xuyunfan/data/mira/shareseq_base_data/rna_data.h5ad")

In [ ]:
rna.var_names_make_unique()
rna.var_names = [x.capitalize() for x in rna.var_names]

In [4]:
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)
sc.pp.highly_variable_genes(rna,n_top_genes=3000)
rna.raw = rna
rna = rna[:, rna.var.highly_variable]
#sc.pp.regress_out(self.adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(rna, max_value=10)
sc.tl.pca(rna,svd_solver='arpack')
sc.pp.neighbors(rna, n_neighbors=10, n_pcs=40)
sc.tl.umap(rna)
sc.tl.leiden(rna,resolution=0.6)

In [5]:
test_data = scripro.Ori_Data(rna,Cell_num=50)

In [ ]:
%%time
test_data.get_positive_marker_gene_parallel(cores=4)

In [ ]:
cellgroup = test_data.adata.obs.loc[:,['new_leiden']]

## Obtain the landscape of each supercell.

In [ ]:
atac_df=atac.to_df()

In [ ]:
atac_df

In [ ]:
scripro.dataframe_to_sparse_tsv(atac_df, 'test.tsv')

In [ ]:
scripro.get_supercell_fragment(cellgroup,'.','./test.tsv',chunksize = 10000000)

In [ ]:
scripro.process_tsv('./supercell_fragment/', 'mm10')

# Calculate TF score

In [ ]:
share_seq = scripro.SCRIPro_Multiome(8,'mm10',test_data)

In [ ]:
%%time
share_seq.cal_ISD_parallel('./bigwig/')

In [ ]:
share_seq.get_P_value_matrix()

In [ ]:
share_seq.get_chip_matrix()

# Calculate TF targets

In [ ]:
share_seq.get_tf()

In [ ]:
share_seq.tf_score

In [ ]:
share_seq.get_tf_target('Gata3')